# Interactive visualization of a robot in pydrake

*Copyright (c) 2018, Toyota Research Institute
All rights reserved.*

See license in `LICENSE`.

## Introduction
This notebook shows how to use Drake and [ipywidgets](https://github.com/jupyter-widgets/ipywidgets) to create an interactive visualization of a robot.

In [1]:
import os.path
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.rigid_body_tree import (
    AddModelInstanceFromUrdfStringSearchingInRosPackages,
    FloatingBaseType,
    RigidBodyTree,
)
from pydrake.lcm import DrakeLcm
from pydrake.systems.framework import BasicVector
from pydrake.multibody.rigid_body_plant import DrakeVisualizer
import numpy as np
import ipywidgets as widgets

## Convenience class for interactive visualization
The following class wraps a `DrakeVisualizer` system and connects it to Jupyter widgets.

In [2]:
class DrakeVisualizerHelper:
    def __init__(self, tree):
        lcm = DrakeLcm()
        self.tree = tree
        self.visualizer = DrakeVisualizer(tree=self.tree, lcm=lcm, enable_playback=True)
        self.x = np.concatenate([robot.getZeroConfiguration(), 
                                 np.zeros(tree.get_num_velocities())])
        self.visualizer.PublishLoadRobot()
        self.draw(self.tree.getZeroConfiguration())
    def draw(self, q = None):
        if q is not None:
            self.x[:self.tree.get_num_positions()] = q
        context = self.visualizer.CreateDefaultContext()
        context.FixInputPort(0, BasicVector(self.x))
        self.visualizer.Publish(context)
        
    def inspect(self, slider_scaling = 1):
        # Setup widgets
        for i in range(robot.number_of_positions()):
            widgets.interact(
                self.__slider_callback,
                slider_value = widgets.FloatSlider(
                    value=slider_scaling * self.x[i],
                    min=slider_scaling * self.tree.joint_limit_min[i],
                    max=slider_scaling * self.tree.joint_limit_max[i],
                    description=self.tree.get_position_name(i)
                ),
                index=widgets.fixed(i),
                slider_scaling=widgets.fixed(slider_scaling)
            )

    def __slider_callback(self, slider_value, index, slider_scaling):
        self.x[index] = slider_value / slider_scaling
        self.draw()        

## Demo

Instantiate a model of the IIWA robot arm along with a visualization helper.

In [3]:
robot = RigidBodyTree("urdf/cassie.urdf", FloatingBaseType.kFixed)
#add_fixed_model(robot, iiwa_file)
vis_helper = DrakeVisualizerHelper(robot)

Draw the arm in a random configuration.

In [4]:
q = robot.getRandomConfiguration()
vis_helper.draw(q)

Show the interactive sliders (with positions scaled to be in degrees).

In [5]:
vis_helper.inspect(np.rad2deg(1))

interactive(children=(FloatSlider(value=9.990923475370707, description=u'hip_roll_left', max=22.517241348641353, min=-15.01149423242757), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=-17.430782158297838, description=u'hip_roll_right', max=15.01149423242757, min=-22.517241348641353), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=-17.16031199268387, description=u'hip_yaw_left', max=22.517241348641353, min=-22.517241348641353), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=15.597620406365847, description=u'hip_yaw_right', max=22.517241348641353, min=-22.517241348641353), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=-33.28329286969375, description=u'hip_pitch_left', max=79.98490820026292, min=-50.019215514920866), Output()), _dom_classes=('widget-interact',))

...

interactive(children=(FloatSlider(value=129.6216521382412, description=u'ankle_joint_right', max=171.88733853924697, min=45.836623610465864), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=-3.8958852993996658, description=u'ankle_spring_joint_left', max=5.729577951308233, min=-5.729577951308233), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=-5.555724549410083, description=u'ankle_spring_joint_right', max=5.729577951308233, min=-5.729577951308233), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=-87.50168455240934, description=u'toe_left', max=-30.02298846485514, min=-139.97358935046012), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=-110.23650507587982, description=u'toe_right', max=-30.02298846485514, min=-139.97358935046012), Output()), _dom_classes=('widget-interact',))